In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- CONFIGURATION ---
# Change this path to point to the specific run you want to analyze (e.g., 70M or 160M)
DETAILS_CSV = "wikipedia/experiments/run_20251221_132209/results/canary_details_full.csv" 
ORIGINAL_CANARIES_CSV = "memorization/canaries.csv" 

# Plotting style
sns.set_theme(style="whitegrid")
plt.rcParams.update({'font.size': 12})

In [42]:
def load_data(details_path, meta_path):
    if not os.path.exists(details_path) or not os.path.exists(meta_path):
        print(f"❌ ERROR: Files not found at:\n{details_path}\n{meta_path}")
        return None

    print("✅ Loading files...")
    df_details = pd.read_csv(details_path)
    df_meta = pd.read_csv(meta_path)

    # Merge to get 'type' column (High/Low Entropy)
    # We merge on 'canary_id'
    df_merged = pd.merge(df_details, df_meta[['canary_id', 'type']], on='canary_id', how='left')
    
    # Check if exact_match exists
    if 'exact_match' not in df_merged.columns:
        print("⚠️ WARNING: 'exact_match' column not found. Filling with 0.")
        df_merged['exact_match'] = 0

    print(f"Merged Data Ready: {len(df_merged)} rows.")
    return df_merged

# Execute Load
df_data = load_data(DETAILS_CSV, ORIGINAL_CANARIES_CSV)
df_data.head()

✅ Loading files...
Merged Data Ready: 1200 rows.


,epoch,canary_id,global_loss_tgt,suffix_loss_tgt,exact_match,split,suffix_loss_ref,global_loss_ref,loss_optimum,mia_score,counterfactual_score,contextual_score,type
0,0,le_f93987,7.568413,6.351475,0,validation,5.950543,7.068214,5.950543,-0.400932,-0.067377,-0.067377,NaN
1,0,he_2fc9c5,3.810425,4.528390,0,train,6.623165,7.991687,6.623165,2.094775,0.316280,0.316280,NaN
2,0,he_487620,3.536453,4.005934,0,train,7.359707,8.466489,6.827364,3.353774,0.455694,0.413253,NaN
3,0,le_d95996,4.706644,3.944786,0,validation,4.103828,4.687080,3.963958,0.159043,0.038755,0.004837,NaN
4,0,he_e6c539,3.696927,3.977993,0,train,7.997181,9.378644,7.786088,4.019188,0.502576,0.489090,NaN


In [43]:
def calculate_dynamic_threshold(scores, fpr_target=0.10):
    """
    Calculates the threshold at the (1-FPR) percentile of the validation scores.
    """
    if len(scores) == 0: return float('inf')
    return np.percentile(scores, (1 - fpr_target) * 100)

def analyze_subset_metrics(df_subset):
    stats = []
    epochs = sorted(df_subset['epoch'].unique())
    
    for epoch in epochs:
        df_epoch = df_subset[df_subset['epoch'] == epoch]
        
        # 1. Split Data
        val_data = df_epoch[df_epoch['split'] == 'validation']
        train_data = df_epoch[df_epoch['split'] == 'train']
        
        # Skip if missing data
        if len(val_data) == 0 or len(train_data) == 0: 
            continue
            
        # 2. Calibrate MIA Threshold using VALIDATION set (Non-members)
        # We want 10% False Positive Rate on non-members
        tau = calculate_dynamic_threshold(val_data['mia_score'].values, fpr_target=0.10)
        
        # 3. Calculate Metrics on TRAINING set (Members)
        
        # A. MIA Recall
        is_memorized = train_data['mia_score'] > tau
        mia_recall = is_memorized.sum() / len(train_data)
        
        # B. Contextual & Counterfactual (Averages)
        avg_ctx = train_data['contextual_score'].mean()
        avg_cf = train_data['counterfactual_score'].mean()
        
        # C. Exact Match (Biderman / Extractability)
        # This is the percentage of training canaries generated verbatim
        if 'exact_match' in train_data.columns:
            extractability = train_data['exact_match'].mean()
        else:
            extractability = 0.0
        
        stats.append({
            'epoch': epoch, 
            'mia_recall': mia_recall, 
            'contextual_score': avg_ctx, 
            'counterfactual_score': avg_cf,
            'extractability': extractability
        })
        
    return pd.DataFrame(stats)

In [44]:
def plot_comparison(df_stats, title):
    if df_stats.empty:
        print(f"⚠️ No data to plot for {title}")
        return

    plt.figure(figsize=(12, 7))
    
    # 1. MIA Recall (Red)
    sns.lineplot(data=df_stats, x='epoch', y='mia_recall', label='MIA Recall', 
                 linewidth=2, color='crimson', alpha=0.6)
    
    # 2. Counterfactual (Green)
    sns.lineplot(data=df_stats, x='epoch', y='counterfactual_score', label='Counterfactual', 
                 linewidth=2, linestyle='-.', color='green', alpha=0.6)
    
    # 3. Contextual (Blue - Your Main Metric)
    sns.lineplot(data=df_stats, x='epoch', y='contextual_score', label='Contextual', 
                 linewidth=3, marker='s', color='royalblue')
    
    # 4. Exact Match (Black/Purple - Biderman)
    sns.lineplot(data=df_stats, x='epoch', y='extractability', label='Exact Match', 
                 linewidth=3, marker='o', color='darkviolet')

    plt.title(title, fontsize=18, fontweight='bold', pad=15)
    plt.xlabel("Training Epochs", fontsize=14)
    plt.ylabel("Score (0.0 - 1.0)", fontsize=14)
    
    # Set limits slightly wider to see points clearly
    plt.ylim(-0.05, 1.05)
    plt.xticks(df_stats['epoch'].unique())
    
    plt.legend(loc='center right', fontsize=12, frameon=True, shadow=True)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [45]:
if df_data is not None:
    # --- ANALYSIS 1: HIGH ENTROPY (Random Codes) ---
    print("--- Analyzing High Entropy Canaries ---")
    df_high = df_data[df_data['type'] == 'high_entropy']
    
    if not df_high.empty:
        stats_high = analyze_subset_metrics(df_high)
        plot_comparison(stats_high, "High Entropy Canaries (Random Codes)")
        # Optional: Print last epoch stats
        print(stats_high.tail(1))
    else:
        print("No High Entropy data found.")

    print("\n" + "="*50 + "\n")



--- Analyzing High Entropy Canaries ---
No High Entropy data found.




In [46]:
# --- ANALYSIS 2: LOW ENTROPY (Natural Language) ---
print("--- Analyzing Low Entropy Canaries ---")
df_low = df_data[df_data['type'] == 'low_entropy']

if not df_low.empty:
    stats_low = analyze_subset_metrics(df_low)
    plot_comparison(stats_low, "Low Entropy Canaries (Natural Language)")
    # Optional: Print last epoch stats
    print(stats_low.tail(1))
else:
    print("No Low Entropy data found.")

--- Analyzing Low Entropy Canaries ---
No Low Entropy data found.
